In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras import layers, models
import tensorflow as tf
import math
import os
import json

import random as python_random

from keras_tuner import HyperModel, HyperParameters
from keras_tuner.tuners import RandomSearch
from keras_tuner.tuners import Hyperband
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.metrics import MeanAbsolutePercentageError

from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D

import time


from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

from funcionesComunes import *

2024-11-27 19:32:57.459485: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-27 19:32:57.462210: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-27 19:32:57.471887: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 19:32:57.488684: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 19:32:57.493653: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 19:32:57.506357: I tensorflow/core/platform/cpu_feature_gu

In [19]:
df_Not_Freeze = pd.DataFrame(columns = ["File", "Time", "TrainMSE", "TrainRMSE", "TrainR2", "TrainMAPE", 
                                "ValidationMSE", "ValidationRMSE", "ValidationR2", "ValidationMAPE",
                                "TestMSE", "TestRMSE", "TestR2", "TestMAPE"]) 

for archivo in os.listdir("RCPMerged"):

    if archivo != "totalMerged.csv":
        
        print("Procesando el archivo: ", archivo)

        # Supongamos que tus datos están en un archivo CSV
        df = pd.read_csv(f'RCPMerged/{archivo}')

        # Ver las primeras filas
        df = codification(df)

        # Normalizamos los datos de crecimiento de los individuos
        df, valorNormalizacion = individualNormalization(df)

        # División adicional para validación
        train_data, val_data, test_data = split_population_individuals(df, train_pct=0.80, val_pct_in_train=0.20, details=False)
        train_data.shape, val_data.shape, test_data.shape

        WINDOWS_SIZE = 3

        # Obtenemos X e y para los datasets de train, val y test 
        X_train, y_train = df_to_X_y_ind_3(train_data, WINDOWS_SIZE)
        X_val, y_val = df_to_X_y_ind_3(val_data, WINDOWS_SIZE)
        X_test, y_test = df_to_X_y_ind_3(test_data, WINDOWS_SIZE)
        print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

        # Cargamos el modelo
        model = models.load_model(f'models/TCNNModels/{archivo.split(".csv")[0]}_model_1.h5', compile=False)

        # Cargamos el json con la arquitectura
        with open(f'models/TCNNModels/{archivo.split(".csv")[0]}_best_models.json') as f:
            parametros = json.load(f)

        parametros = parametros[0]
        batch_size = parametros['batch_size']

        # Obtener el optimizador según los valores del JSON
        learning_rate = parametros['learning_rate']
        optimizer_name = parametros['optimizer']
        batch_size_json = parametros['batch_size']  

        if optimizer_name == 'rmsprop':
            opt = RMSprop(learning_rate=learning_rate)
        elif optimizer_name == 'adam':
            opt = Adam(learning_rate=learning_rate)
        elif optimizer_name == 'sgd':
            opt = SGD(learning_rate=learning_rate)
        else:
            raise ValueError(f"Optimizador {optimizer_name} no reconocido")

        # Compilar el modelo con los hiperparámetros del JSON
        model.compile(optimizer=opt,
                    loss=MeanSquaredError(),
                    metrics=[
                        MeanSquaredError(name='mse'),
                        RootMeanSquaredError(name='rmse'),
                        MeanAbsolutePercentageError(name='mape')
                    ])
        
        model.save(f'models/TCNN/model_{archivo.split("_merged")[0]}.keras')
                
        # # Inicio del entrenamiento 
        # start_time = time.time()

        # # Entrenar el modelo
        # historyTCMM = model.fit(X_train, y_train, epochs=200, batch_size=batch_size,
        #             validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=10)])
        
        # # Fin del entrenamiento 
        # end_time = time.time()

        # Realizar predicciones y calcular métricas para el conjunto de entrenamiento
        predictions_train = predictionForIndividuals(X_train, y_train, model, batch_size)
        predictions_train["PredictionsDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_train["ActualDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        train_mse = mean_squared_error(predictions_train["ActualDenormalize"],predictions_train["PredictionsDenormalize"])
        train_rmse = np.sqrt(train_mse)
        train_mape = (np.sum(np.abs(predictions_train["PredictionsDenormalize"] - predictions_train["ActualDenormalize"])) / np.sum(np.abs(predictions_train["ActualDenormalize"]))) * 100
        train_r2 = r2_score(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])

        # Realizar predicciones y calcular métricas para el conjunto de validación
        predictions_val = predictionForIndividuals(X_val, y_val, model, batch_size)
        predictions_val["PredictionsDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_val["ActualDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        val_mse = mean_squared_error(predictions_val["ActualDenormalize"],predictions_val["PredictionsDenormalize"])
        val_rmse = np.sqrt(val_mse)
        val_mape = (np.sum(np.abs(predictions_val["PredictionsDenormalize"] - predictions_val["ActualDenormalize"])) / np.sum(np.abs(predictions_val["ActualDenormalize"]))) * 100
        val_r2 = r2_score(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])

        # Realizar predicciones y calcular métricas para el conjunto de prueba
        predictions_test = predictionForIndividuals(X_test, y_test, model, batch_size)
        predictions_test["PredictionsDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_test["ActualDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        test_mse = mean_squared_error(predictions_test["ActualDenormalize"],predictions_test["PredictionsDenormalize"])
        test_rmse = np.sqrt(test_mse)
        test_mape = (np.sum(np.abs(predictions_test["PredictionsDenormalize"] - predictions_test["ActualDenormalize"])) / np.sum(np.abs(predictions_test["ActualDenormalize"]))) * 100
        test_r2 = r2_score(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])

        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Train): {train_mse}, {train_rmse}, {train_r2}, {train_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Val): {val_mse}, {val_rmse}, {val_r2}, {val_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Test): {test_mse}, {test_rmse}, {test_r2}, {test_mape}")

        # Guardamos los datos calculados
        df_Not_Freeze.loc[len(df_Not_Freeze)] = [archivo, -999, train_mse, train_rmse, train_r2, train_mape, val_mse, val_rmse, val_r2, val_mape, test_mse, test_rmse, test_r2, test_mape]

df_Not_Freeze.to_csv("resultsTransfer/TCNN/Global/resultados_Not_Transfer_MultipleLayers.csv", index=False)

Procesando el archivo:  Abies spectabilis_merged.csv
(56618, 4, 43) (56618,) (15689, 4, 43) (15689,) (19637, 4, 43) (19637,)


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries i

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 16.992359388702223, 4.12217896126578, 0.8509182621784479, 20.203425817542623
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 17.620423648187227, 4.197668834982963, 0.83944895271168, 20.71440540072978
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 17.607770755828092, 4.196161431097247, 0.8694162656119522, 19.953329034632635
Procesando el archivo:  Picea smithiana_merged.csv
(19933, 4, 43) (19933,) (5863, 4, 43) (5863,) (7272, 4, 43) (7272,)


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries i

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 61.76854832551392, 7.859296935827906, 0.8189827141441706, 22.04134793491326
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 60.540515090709896, 7.780778565844802, 0.825296136273859, 21.547447014180012
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 53.27096026861117, 7.298695792305032, 0.8201062538023935, 22.928274612585533
Procesando el archivo:  Abies pindrow_merged.csv
(9042, 4, 43) (9042,) (2815, 4, 43) (2815,) (3441, 4, 43) (3441,)


...

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 2.0799432422658484, 1.4422008328474396, 0.8340556977068581, 25.382730243214404
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 0.6869648424777913, 0.8288334226355687, 0.6951776461269701, 27.911098051096356
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 3.785924904393789, 1.9457453339000428, 0.8002902377543752, 27.862424286092452
Procesando el archivo:  Juniperus recurva_merged.csv
(3188, 4, 43) (3188,) (810, 4, 43) (810,) (1082, 4, 43) (1082,)


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries i

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 3.5697852525939493, 1.889387533724606, 0.8581499337869172, 21.260826990440645
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 3.4419489988301026, 1.8552490395712653, 0.8750179564882148, 20.874614307450493
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 1.1650738876307616, 1.079385884487453, 0.876138374718838, 16.470256674396094
Procesando el archivo:  Juniperus spp. _merged.csv
(10218, 4, 43) (10218,) (3018, 4, 43) (3018,) (3900, 4, 43) (3900,)


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries i

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 6.93322346903177, 2.633101492353033, 0.886923875565214, 20.061323307284418
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 2.780104229896562, 1.6673644562292198, 0.8586824025222065, 21.922721502898526
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 8.382794264934477, 2.8953055564023766, 0.9197790501192903, 19.232067977876355


## Transfer Learning

In [3]:
# Leemos los diferentes modelos
for model in os.listdir("models/TCNN"):

    # Creamos un df que contiene los resultados de los modelos freeze de cada especie
    df_Freeze = pd.DataFrame(columns = ["File", "Time", "TrainMSE", "TrainRMSE", "TrainR2", "TrainMAPE", 
                                        "ValidationMSE", "ValidationRMSE", "ValidationR2", "ValidationMAPE",
                                        "TestMSE", "TestRMSE", "TestR2", "TestMAPE"])

    print(f'Procesando el modelo: {model}')

    # Para cada archivo 
    for archivo in os.listdir("RCPMerged"):

        # Comprobamos que el modelo no sea del archivo a procesar
        archivoProcesar = archivo.split("_merged")[0]
        modeloProcesar = model.split("model_")[1].split(".keras")[0]
        print(archivoProcesar, modeloProcesar)

        # Lanzamos la ejecución siempre que no sea el mismo archivo o el global
        if archivoProcesar != modeloProcesar and archivo != "totalMerged.csv":

            # Cargar el archivo CSV con punto y coma como delimitador
            df = pd.read_csv(f'RCPMerged/{archivo}')
            df = df[~df["nametag"].str.startswith("INDI005")]

            # Codificamos, normalización y split de datos
            df = codification(df)
            print(df.shape)

            df, valorNormalizacion = individualNormalization(df)
            print(f"SE HA NORMALIZADO EL ARCHIVO: {archivo}")

            temp_df = df
            train_data, val_data, test_data = split_population_individuals(temp_df, train_pct=0.80, val_pct_in_train=0.20, details=False)
            train_data.shape, val_data.shape, test_data.shape

            # Obtenemos X e y para los datasets de train, val y test 
            WINDOWS_SIZE = 3
            X_train, y_train = df_to_X_y_ind_3(train_data, WINDOWS_SIZE)
            X_val, y_val = df_to_X_y_ind_3(val_data, WINDOWS_SIZE)
            X_test, y_test = df_to_X_y_ind_3(test_data, WINDOWS_SIZE)
            print(X_train.shape, X_test.shape, X_val.shape)

            # Cargamos el modelo global
            modelLSTM = tf.keras.models.load_model(f'models/TCNN/{model}')
            modelLSTMFreeze = modelLSTM

            # Obtener el optimizador del modelo
            optimizer = modelLSTMFreeze.optimizer

            # Obtenemos el valor de batch size
            with open(f'arquitecturaModelosJSON/TCNN/{archivo.split(".csv")[0]}_best_models.json') as f:
                parameters = json.load(f)

            batch_size_LSTM = parameters[0]["batch_size"]

            # Indicamos el numero de layers a entrenar
            NUM_TRAINABLE = 1

            numTCNN_layers = sum(1 for layer in modelLSTMFreeze.layers if isinstance(layer, (Conv1D)))

            numFreezeLayers = numTCNN_layers - NUM_TRAINABLE # Congelamos todas menos la útlima capa

            numberLayersFreezed = 0

            # Recorre las capas en orden inverso
            for i, layer in enumerate(modelLSTMFreeze.layers):

                if isinstance(layer, (Conv1D)):
                    numberLayersFreezed +=1
            
                if isinstance(layer, (Conv1D, BatchNormalization, MaxPooling1D)) and numFreezeLayers>=numberLayersFreezed:
                    
                    layer.trainable = False
                
                print(layer, layer.trainable)


            # Compilamos el modelo con los nuevos datos
            modelLSTMFreeze.compile(optimizer=optimizer, loss=MeanSquaredError(), metrics=[
                                        MeanSquaredError(name='mse'),
                                        RootMeanSquaredError(name='rmse'),
                                        MeanAbsolutePercentageError(name='mape')
                                    ])

            # Comienza a medir el tiempo de entrenamiento
            start_time = time.time()

            historyLSTMTransfer = modelLSTMFreeze.fit(X_train, y_train, epochs=200, batch_size=batch_size_LSTM,
                                    validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=0)

            # Finaliza la medición del tiempo de entrenamiento
            end_time = time.time()
            print(f"[MODELO CONGELADO]: {end_time-start_time}")

            # Realizar predicciones y calcular métricas para el conjunto de entrenamiento
            predictions_train = predictionForIndividuals(X_train, y_train, modelLSTMFreeze, batch_size_LSTM)
            predictions_train["PredictionsDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
            predictions_train["ActualDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

            train_mse = mean_squared_error(predictions_train["ActualDenormalize"],predictions_train["PredictionsDenormalize"])
            train_rmse = np.sqrt(train_mse)
            train_mape = (np.sum(np.abs(predictions_train["PredictionsDenormalize"] - predictions_train["ActualDenormalize"])) / np.sum(np.abs(predictions_train["ActualDenormalize"]))) * 100
            train_r2 = r2_score(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])

            # Realizar predicciones y calcular métricas para el conjunto de validación
            predictions_val = predictionForIndividuals(X_val, y_val, modelLSTMFreeze, batch_size_LSTM)
            predictions_val["PredictionsDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
            predictions_val["ActualDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

            val_mse = mean_squared_error(predictions_val["ActualDenormalize"],predictions_val["PredictionsDenormalize"])
            val_rmse = np.sqrt(val_mse)
            val_mape = (np.sum(np.abs(predictions_val["PredictionsDenormalize"] - predictions_val["ActualDenormalize"])) / np.sum(np.abs(predictions_val["ActualDenormalize"]))) * 100
            val_r2 = r2_score(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])

            # Realizar predicciones y calcular métricas para el conjunto de prueba
            predictions_test = predictionForIndividuals(X_test, y_test, modelLSTMFreeze, batch_size_LSTM)
            predictions_test["PredictionsDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
            predictions_test["ActualDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

            test_mse = mean_squared_error(predictions_test["ActualDenormalize"],predictions_test["PredictionsDenormalize"])
            test_rmse = np.sqrt(test_mse)
            test_mape = (np.sum(np.abs(predictions_test["PredictionsDenormalize"] - predictions_test["ActualDenormalize"])) / np.sum(np.abs(predictions_test["ActualDenormalize"]))) * 100
            test_r2 = r2_score(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])

            print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Train): {train_mse}, {train_rmse}, {train_r2}, {train_mape}")
            print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Val): {val_mse}, {val_rmse}, {val_r2}, {val_mape}")
            print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Test): {test_mse}, {test_rmse}, {test_r2}, {test_mape}")

            # Guardamos los datos calculados
            df_Freeze.loc[len(df_Freeze)] = [archivo, end_time-start_time,train_mse, train_rmse, train_r2, train_mape, val_mse, val_rmse, val_r2, val_mape, test_mse, test_rmse, test_r2, test_mape]

    df_Freeze.to_csv(f'resultsTransfer/TCNN/resultadosTrasnfer_{model.split("model_")[1].split(".keras")[0]}.csv', index=False)

Procesando el modelo: model_Pinus gerardiana.keras
Abies spectabilis Pinus gerardiana
(96429, 77)
SE HA NORMALIZADO EL ARCHIVO: Abies spectabilis_merged.csv
(56618, 4, 43) (19637, 4, 43) (15689, 4, 43)


I0000 00:00:1732732419.918038 3481384 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732732419.918336 3481384 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-27 19:33:39.922852: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/

<Conv1D name=conv1d_6, built=True> False
<BatchNormalization name=batch_normalization_6, built=True> False
<MaxPooling1D name=max_pooling1d_5, built=True> False
<Conv1D name=conv1d_7, built=True> False
<BatchNormalization name=batch_normalization_7, built=True> False
<MaxPooling1D name=max_pooling1d_6, built=True> False
<Conv1D name=conv1d_8, built=True> False
<BatchNormalization name=batch_normalization_8, built=True> False
<MaxPooling1D name=max_pooling1d_7, built=True> False
<Dropout name=dropout_1, built=True> True
<Conv1D name=conv1d_9, built=True> False
<BatchNormalization name=batch_normalization_9, built=True> False
<MaxPooling1D name=max_pooling1d_8, built=True> False
<Conv1D name=conv1d_10, built=True> True
<BatchNormalization name=batch_normalization_10, built=True> True
<GlobalAveragePooling1D name=global_average_pooling1d_1, built=True> True
<Dense name=dense_2, built=True> True
<Dense name=dense_3, built=True> True
[MODELO CONGELADO]: 294.5899293422699


/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 84.26563638518476, 9.179631604001587, 0.26069904575495195, 49.73059785188782
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 82.28013775596257, 9.070839969703059, 0.2502925836799571, 49.03547043552493
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 87.8525893113706, 9.37297121042045, 0.3484627130245297, 48.61058063814436
Picea smithiana Pinus gerardiana
(34476, 77)
SE HA NORMALIZADO EL ARCHIVO: Picea smithiana_merged.csv
(19933, 4, 43) (7272, 4, 43) (5863, 4, 43)
<Conv1D name=conv1d_6, built=True> False
<BatchNormalization name=batch_normalization_6, built=True> False
<MaxPooling1D name=max_pooling1d_5, built=True> False
<Conv1D name=conv1d_7, built=True> False
<BatchNormalization name=batch_normalization_7, built=True> False
<MaxPooling1D name=max_pooling1d_6, built=True> False
<Conv1D name=conv1d_8, built=True> False
<BatchNormalization name=batch_normalization_8, built=True> False
<MaxPooling1D name=max_pooling1d_7, built=True> False
<Dropout name=dropo

...

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 3.5069591967445977, 1.8726876933286547, 0.8606464089392143, 20.64740558254886
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 3.5298396689163685, 1.878786754508443, 0.8718265217642442, 21.17816954691006
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 1.4136670534566245, 1.1889773141051196, 0.8497098761661873, 17.52409005215793
totalMerged.csv Picea smithiana
Juniperus spp.  Picea smithiana
(17976, 77)
SE HA NORMALIZADO EL ARCHIVO: Juniperus spp. _merged.csv
(10218, 4, 43) (3900, 4, 43) (3018, 4, 43)
<Conv1D name=conv1d_3, built=True> False
<BatchNormalization name=batch_normalization_3, built=True> False
<MaxPooling1D name=max_pooling1d_2, built=True> False
<Conv1D name=conv1d_4, built=True> False
<BatchNormalization name=batch_normalization_4, built=True> False
<MaxPooling1D name=max_pooling1d_3, built=True> False
<Conv1D name=conv1d_5, built=True> False
<BatchNormalization name=batch_normalization_5, built=True> False
<MaxPooling1D name=max_pooling1d_4, 

/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 58 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


[MODELO CONGELADO]: 20.75176477432251


/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 6.9787967326388864, 2.6417412312031785, 0.8861806068606128, 21.08782209118952
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 3.120686803397005, 1.7665465754961018, 0.8413699900909368, 23.819064927517175
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 9.49224095309818, 3.080948060759574, 0.9091619617888759, 21.434884962057648
